*Note: all those exercises should be done using python with pyccel openmp*

## Exercise 1 Hello World

1. Write an OpenMP program displaying the number of threads used for the execution and the rank of each of the threads.
3. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
>> Python hello.py
Hello from the rank 2 thread
Hello from the rank 0 thread
Hello from the rank 3 thread
Hello from the rank 1 thread
Hello from the rank 4 thread
Parallel execution of hello_world with 4 threads
```
*Note that the output order maybe different*

In [1]:
%%file hello.py
if __name__ == "__main__":
    from pyccel.stdlib.internal.openmp import omp_get_num_threads, omp_get_thread_num
    #$ omp parallel
    print("Hello from the rank", omp_get_thread_num(), "thread")
    num_threads = omp_get_num_threads()

    #$ omp end parallel
    print("Parallel execution of hello_world with, ", num_threads, "threads")

Writing hello.py


In [5]:
!pyccel hello.py --language=c --openmp; ./hello

/home/rakati/Teaching_dev/MSD/hpc-env/lib/python3.8/site-packages/pyccel/codegen/compiling/compilers.py:401: UserWarning: /home/rakati/Teaching_dev/MSD/hpc/day02/assignments/__pyccel__/prog_hello.c: In function ‘main’:
/home/rakati/Teaching_dev/MSD/hpc/day02/assignments/__pyccel__/prog_hello.c:11:22: warning: format ‘%ld’ expects argument of type ‘long int’, but argument 3 has type ‘int’ [-Wformat=]
   11 |         printf("%s %ld %s\n", "Hello from the rank", omp_get_thread_num(), "thread");
      |                    ~~^                               ~~~~~~~~~~~~~~~~~~~~
      |                      |                               |
      |                      long int                        int
      |                    %d

  warnings.warn(UserWarning(err))
Hello from the rank 1 thread
Hello from the rank 7 thread
Hello from the rank 2 thread
Hello from the rank 4 thread
Hello from the rank 6 thread
Hello from the rank 0 thread
Hello from the rank 5 thread
Hello from the rank 3 thr

## Exercise 2 Matrix product

Considering the following code for matrix product:

1. Using pyccel, epyccelize the `matrix_prod` function, and time the execution of epyccelized function.
2. Insert the appropriate OpenMP directives and analyse the code performance.
3. Test the loop iteration repartition modes (`STATIC`, `DYNAMIC`, `GUIDED`) and vary the chunk sizes.


In [14]:
def matrix_prod(A:'real[:,:]', B:'real[:,:]', C:'real[:,:]', N:int, M:int):
    from pyccel.stdlib.internal.openmp import omp_get_num_threads
    #$ omp parallel
    nb_threads = omp_get_num_threads()
    #$ omp for schedule(runtime) nowait
    for i in range( M ):
        for j in range( N ):
            A[ i, j ] = (i + 1) + (j + 1)

    #$ omp for schedule(runtime) nowait
    for i in range( N ):
        for j in range( M ):
            B[ i, j ] = (i + 1) - (j + 1)

    #$ omp for schedule(runtime)
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0
    # ...

    # Matrix Production
    #$ omp for schedule(runtime) nowait
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]

    #$ omp end parallel
    print("Execution of Matrix production in parallele with",nb_threads, "threads")


if __name__ == "__main__":
    import numpy as np
    import time
    import math

    M = 20
    N = 20

    # Declare Matrices
    A = np.empty((M, N), dtype=np.double)
    B = np.empty((N, M), dtype=np.double)
    C = np.empty((M, M), dtype=np.double)

    from pyccel.epyccel import epyccel
    c_matrix_prod = epyccel(matrix_prod, language='fortran', accelerators=['openmp'])

    # Start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time

    # -------------------- computing ------------------
    c_matrix_prod(A, B, C, M, N)
    # ------------------ End computing ----------------

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # Print result
    print(
      "   Value of M and N           :", M, N,      "\n",
	  "   Temps elapsed              :", t_elapsed, "\n",
	  "   Temps CPU                  :", tcpu,      "\n",
      "   Partiel results            :", C[ 1, 1 ], C[ 2, 2 ], "...", C[ M-3, M-3 ], C[ M-2, M-2 ], "\n"
    )


   Value of M and N           : 20 20 
    Temps elapsed              : 0.0009012222290039062 
    Temps CPU                  : 0.0 
    Partiel results            : 2790.0 2690.0 ... -3610.0 -4350.0 

 Execution of Matrix production in parallele with                     8 threads


## Exercise 3 Jacobi method

Considering the following code for a general linear system solver:

$$ A \times x = b $$

using the Jacobi iterative method.
In this exercice, you must solve the system in parallel


In [3]:
def jacobi(a:'double[:,:]', b:'double[:]', x:'double[:]', n:int, eps:'double'):
    from pyccel.stdlib.internal.openmp import omp_get_num_threads
    #$ omp parallel
    nb_tasks = omp_get_num_threads()
    print("Execution of Jacobi in parallel with", nb_tasks, "threads")
    #$ omp end parallel
    # Jacobi method resolution
    import numpy as np
    import math
    x_courant = np.empty(n, dtype=np.double)
    iteration = 0
    while(1):
        iteration += 1

        #$ omp parallel for schedule(runtime)
        for i in range(n):
            x_courant[i] = 0
            for j in range(i):
                x_courant[i] += a[ j, i ] * x[j]

            for j in range(i + 1, n):
                x_courant[i] += a[ j, i ] * x[j]

            x_courant[i] = (b[i] - x_courant[i]) / a[ i, i ]

        # Convergence test
        absmax = 0.
        for i in range(n):
            curr = math.fabs(x[i] - x_courant[i])
            if curr > absmax:
                absmax = curr
        stand = absmax / n
        if stand <= eps or iteration > n:
            break

        # copy x_courant into x
        for i in range(n):
            x [i] = x_courant[i]
    return stand, iteration


if __name__ == "__main__":

    import numpy as np
    import time, random, math, sys

    # Matrix default dimension
    n = 20
    diag = 20

    # Initialize of the matrices
    np.random.seed(421)  # for reproductibles results
    a = np.random.uniform(0, 1, size=(n, n))
    b = np.random.uniform(0, 1, size=n)

    # We strengthen the main diagonal of the matrix
    for i in range(n):
        a[ i, i ] += diag

    # Initial solution
    x = np.ones(n, dtype=np.double)

    # start CPU timing
    cpu_0 = time.process_time()

    # start Wall timing
    elapsed_0 = time.time()

    # Pyccelize jacobi function
    from pyccel.epyccel import epyccel
    p_jacobi = epyccel(jacobi, language='c', accelerators=['openmp'])

    eps = sys.float_info.epsilon

    # -------------------- computing ------------------
    stand, iteration = p_jacobi(a, b, x, n, eps)
    # ------------------ End computing ----------------

    # CPU time
    cpu_1 = time.process_time()
    cpu = cpu_1 - cpu_0

    # Wall timing
    elapsed_1 = time.time()
    elapsed = elapsed_1 - elapsed_0

    print(
        "System size    :", n,          '\n',
        "Iterations     :", iteration,  '\n',
        "Stand          :", stand,      '\n',
        "Elapsed Time   :", elapsed,    '\n',
        "CPU time       :", cpu,        '\n'
    )

System size    : 20 
 Iterations     : 21 
 Stand          : 2.9100049961372497e-08 
 Elapsed Time   : 2.1162259578704834 
 CPU time       : 0.125 



## Exercise 4 Calculation of π

The aim of this exercise is to calculate π by numerical integration knowing that Considering the following code for matrix product: $\int_{0}^{1} \frac{4}{1 + x^2} = {\pi} $

The following program is for calculating the value of
π by the rectangle method (mid-point). Let $f(x) = \frac{4}{1 + x^2} $
2 be the function to integrate, N and $ h = \frac{1}{N} $ (respectively) the number of points, and the discretization width on the integration
interval $[0, 1]$.

This exercice can be parallelized in three different ways (i.e. using different **OpenMP** directives for each version). Analyse the performance of the three codes, then optimise the least efficient versions (without changing the type of OpenMP directives used), in order to obtain the same performance for the three parallelized versions.

In [24]:
%%file pi.py
def f(a:'double'):
    return 4.0 / (1. + a * a)

def pi(n:int, h:'double'):


    from pyccel.stdlib.internal.openmp import omp_get_num_threads
    #$ omp parallel
    nb_tasks = omp_get_num_threads()
    #$ omp end parallel
    print("Execution of PI in parallel with", nb_tasks,"threads")

    for k in range(100):
        Pi_calc = 0.
        #$ omp parallel for reduction(+: Pi_calc) schedule(runtime)
        for i in range(n):
            x = h * (i + 0.5)
            Pi_calc += f(x)

        Pi_calc = h * Pi_calc
    return Pi_calc

Writing pi.py


In [25]:
import time, math

if __name__ == '__main__':
    #                  __
    #  But : calcul de || par la methode des rectangles (point milieu).
    #
    #                   / 1
    #                  |       4            __
    #                  |   ----------  dx = ||
    #                  |    1 + x**2
    #                 / 0


    n = 300

    # Length of the integration interval
    h = 1.0 / n

    from pyccel.epyccel import epyccel
    import pi
    pyc_pi = epyccel(pi, language='c', accelerators=['openmp'])

    # start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time

    # -------------------- computing ------------------

    Pi_calc = pyc_pi.pi(n, h)

    # ------------------ End computing ----------------

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # deviation between the estimated value and the calculated value of Pi
    Pi_estime = math.acos(-1)
    deviat = math.fabs(Pi_estime - Pi_calc)

    # Print result
    print(
      "   Number of intervals        :", n,         "\n",
	  "   | Pi_estime - Pi_calcule | :", deviat,    "\n",
	  "   Temps elapsed              :", t_elapsed, "\n",
	  "   Temps CPU                  :", tcpu,      "\n",
    )

   Number of intervals        : 300 
    | Pi_estime - Pi_calcule | : 9.259259270422149e-07 
    Temps elapsed              : 0.0015625953674316406 
    Temps CPU                  : 0.0 



## Exercise 5 The conjugate gradient method

The program contained in the gradient conjugue.f90 file solves a symmetric linear system

$$ A \times x = b $$

using the preconditioned conjugate gradient method. In Python, this program can be parallelized primarily by using the WORKSHARE constructions.
1. After introducing the appropriate OpenMP directives, analyse the code performance.
2. What are your conclusions about the effectiveness of the WORKSHARE directive ?
3. Optimise the parallel version of the code by slightly modifying the source code in order to avoid using the WORKSHARE directive in places where it is problematic.

In [16]:
def cg(A:'double[:,:]', B:'double[:]', n:int):
#$ omp parallel

    # On symmetrise la matrice
  #$ omp workshare
    for i in range(1, n):
        for j in range(n - 1):
            A[ i, j ] = A[ j, i ]

    print(A)
    # We strengthen the main diagonal
    for i in range(n):
        A[ i, i ] += 800.0

    pre = np.empty((n), dtype=np.double)
    # We choose the Jacobi preconditioning = Diagonal (a)
    for i in range(n):
        pre[i] = A[ i, i ]

    # Initial solution
    x = np.ones((n), dtype=np.double)

    # Initial ecart
    r = B[:] - np.matmul(A, x)
  #$ omp end workshare

    # Resolution using gradient conjugue methode
    iteration = 0
    while True:
      #$ omp single
        rho = 0.0
        gamma = 0.0
        iteration += 1
      #$ omp end single
      #$ omp workshare
#         not completed !!!!!!!!!

      #$ omp end workshare

#$ omp end parallel


if __name__ == '__main__':
    import numpy as np
    n = 4201

    # Initialize Matrices
    A = np.random.uniform(0, 1, size=(n, n))
    B = np.random.uniform(0, 1, size=(n, n))
    cg(A, B, n)

[[0.80595766 0.98699575 0.98699469 ... 0.48088396 0.70539741 0.35324892]
 [0.98699575 0.64720747 0.31109875 ... 0.43588759 0.54116159 0.18821353]
 [0.98699469 0.31109875 0.13036537 ... 0.49731452 0.60446757 0.30887318]
 ...
 [0.48088396 0.43588759 0.49731452 ... 0.97571607 0.42813009 0.35654696]
 [0.70539741 0.54116159 0.60446757 ... 0.42813009 0.65288483 0.44419889]
 [0.35324892 0.18821353 0.30887318 ... 0.35654696 0.44419889 0.0127382 ]]


NameError: name 'z' is not defined

## Exercise 6 Reduction of an array

The given program contained in the `reduction_tab.py` file is extracted from a chemistry code. It reduces a three-dimensional array into a vector. The aim of this exercise is to parallelize this calculation kernel without changing the loop order in the provided code (i.e. k,j,i)

1. Analyse the data-sharing attributes of the variables and adapt the source code so that the K outermost loop is parallelized.
2. Compare the performance obtained by using the thread/core binding default execution on Ada and by using scatter binding. Suggest an explanation for the poor performance of the latter.
3. Optimise the source code for the scatter mode with taking into account the memory affinity. Why does this third series of executions give the best performance 

In [18]:
%%file reduction_tab.py

def threads_num():
    from pyccel.stdlib.internal.openmp import omp_get_num_threads
    #$ omp parallel
    nb_taches = omp_get_num_threads()
    #$ omp end parallel
    return nb_taches

def initialize_table(tab:'double[:,:,:]', nmolec:int, n:int, nmol:int):
    # Initialisation du tableau
    # First-touch pour garantir un fonctionnement optimal sur les syst�mes NUMA
    #$ omp for schedule(static)
    for k in range(nmolec):
        for j in range(n):
            for i in range(nmol):
                tab[ i,j,k ] = i + j + k

def reduction(tab:'double[:,:,:]', tab1:'double[:]', tab2:'double[:]', nmolec:int, nmol:int, n:int):
    #$ omp parallel private(tab1)
    #$ omp for schedule(static) reduction(+:tab2)
    for k in range(nmolec):
        tab1[:nmol] = 0
        for j in range(n):
            for i in range(nmol):
                tab1[i] = tab1[i] + tab[i,j,k]
        tab2[:nmol] = tab2[:nmol] + 2 * tab1[:nmol]
    #$ omp end parallel

Writing reduction_tab.py


In [19]:
from os import error

if __name__ == '__main__':
    import numpy as np
    import time

    # default dimension
    nmolec = 100
    nmol   = 100
    n      = 10

    tab     = np.empty((nmol, n, nmolec), dtype=np.double)
    tab1    = np.empty((nmol), dtype=np.double)
    tab2    = np.zeros((nmol), dtype=np.double)

    # pyccelize reduction tab module
    from pyccel.epyccel import epyccel
    import reduction_tab
    
    reduction = epyccel(reduction_tab, language='fortran', accelerators=['openmp'])
    
    print("Execution of reduction table in parallele with", reduction.threads_num, "threads")

    # initialize tab 
    reduction.initialize_table(tab, nmol, n, nmolec)

    # start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time

    # -------------------- computing ------------------
    reduction.reduction(tab, tab1, tab2, nmolec, nmol, n)
    # ------------------ End computing ----------------

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # verification of results
    tab2c = np.zeros((nmol), dtype=np.double)
    for k in range(nmolec):
        tab1c = np.zeros((nmol), dtype=np.double)
        for j in range(n):
            for i in range(nmol):
                tab1c[i] = tab1c[i] + tab[i, j, k]
        tab2c [:] = tab2c[:nmol] + 2 * tab1c[:]
    err = np.max(np.abs(tab2c - tab2) / np.abs(tab2c))
    print(
        "Temps elapsed  :", t_elapsed   , "\n",
        "Temps CPU      :", tcpu        , "\n",
        "Erreur relative:", err         , "\n"
    )

Execution of reduction table in parallele with <built-in function threads_num> threads
Temps elapsed  : 0.0009007453918457031 
 Temps CPU      : 0.0 
 Erreur relative: 0.0 



# Exercise 7 Matrix product by the Strassen algorithm

Considering the following code that calculates the matrix product :

$$C = A \times B$$

by using Strassen’s recursive algorithm.

In this exercise, you must :
1. Analyse and parallelize the code by using OpenMP tasks.
2. Measure the code performance and plot the speedup curves obtained.

In [21]:
import time, math
import numpy as np

def threads_num():
    from pyccel.stdlib.internal.openmp import omp_get_num_threads
    #$ omp parallel
    nb_taches = omp_get_num_threads()
    #$ omp end parallel
    return nb_taches


def strassen_multiply(A:'double[:, :]', B:'double[:, :]', C:'double[:, :]', n:int):
    import numpy as np
    if n & 1 != 0  or n < 128:
        C[:] = np.matmul(A, B)
    else:
        n2 = n // 2
        A11 = A[ 0:n2, 0:n2 ]
        A21 = A[ n2:n, 0:n2 ]
        A12 = A[ 0:n2, n2:n ]
        A22 = A[ n2:n, n2:n ]
        B11 = B[ 0:n2, 0:n2 ]
        B21 = B[ n2:n, 0:n2 ]
        B12 = B[ 0:n2, n2:n ]
        B22 = B[ n2:n, n2:n ]

        Q1 = np.empty((n2, n2), dtype=np.double)
        Q2 = np.empty((n2, n2), dtype=np.double)
        Q3 = np.empty((n2, n2), dtype=np.double)
        Q4 = np.empty((n2, n2), dtype=np.double)
        Q5 = np.empty((n2, n2), dtype=np.double)
        Q6 = np.empty((n2, n2), dtype=np.double)
        Q7 = np.empty((n2, n2), dtype=np.double)
        
        #$ omp task shared(A11,A22,B11,B22,Q1,n) 
        strassen_multiply(A11+A22, B11+B22, Q1, n2)
        #$ omp end task
        #$ omp task shared(A21,A22,B11,Q2,n)
        strassen_multiply(A21+A22, B11, Q2, n2)
        #$ omp end task
        #$ omp task shared(A11,B12,B22,Q3,n) 
        strassen_multiply(A11, B12-B22, Q3, n2)
        #$ omp end task
        #$ omp task shared(A22,B11,B21,Q4,n) 
        strassen_multiply(A22, -B11+B21, Q4, n2)
        #$ omp end task
        #$ omp task shared(A11,A12,B22,Q5,n) 
        strassen_multiply(A11+A12, B22, Q5, n2)
        #$ omp end task
        #$ omp task shared(A11,A21,B11,B12,Q6,n) 
        strassen_multiply(-A11+A21, B11+B12, Q6, n2)
        #$ omp end task
        #$ omp task shared(A12,A22,B21,B22,Q7,n)
        strassen_multiply(A12-A22, B21+B22, Q7, n2)
        #$ omp end task
        #$ omp taskwait
        C[   :n2,   :n2 ] = Q1+Q4-Q5+Q7
        C[ n2: n,   :n2 ] = Q2+Q4
        C[   :n2, n2:n  ] = Q3+Q5
        C[ n2:n , n2:n  ] = Q1+Q3-Q2+Q6


def calcul_erreur(n:int, A:'double[:, :]', B:'double[:, :]', C:'double[:, :]'):
    D = np.matmul(A, B) 
    error = 0.0
    for i in range(n):
        for j in range(n):
            e = C[ i, j ] - D[ i, j ]
            error += e * e

    error = math.sqrt(error) / n
    return error

if __name__ == '__main__':
    n = 256

    # Initialize Matrices
    np.random.seed(421)
    A = np.random.uniform(0, 1, (n, n))
    B = np.random.uniform(0, 1, (n, n))
    C = np.empty((n,n), dtype=np.double)

    from pyccel.epyccel import epyccel
    strassen = epyccel(strassen_multiply, accelerators=['openmp'])
    num_threads = epyccel(threads_num, accelerators=['openmp'])
    print("Execution of Strassen in parallele with", num_threads(), "threads")
    
    
    # Start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time
    # -------------------- computing ------------------
    # Calcul C=A*B par la methode recursive de Strassen
    strassen(A, B, C, n)
    # ------------------ end computing ----------------
    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0
    # Print result
    print(
      "   Erreur        :", calcul_erreur(n, A, B, C),  "\n",
	  "   Temps elapsed :", t_elapsed,                  "\n",
	  "   Temps CPU     :", tcpu,                       "\n",
    )


   Erreur        : 4.962553327910702e-14 
    Temps elapsed : 0.0027666091918945312 
    Temps CPU     : 0.0 

